<a href="https://colab.research.google.com/github/head4ths/CEI/blob/master/part2_bert%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D_%EB%AA%A8%EB%8D%B8%ED%95%99%EC%8A%B5%EB%B0%8F%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 비고

In [1]:
"""
원본은 colab에서 관리 (이름 안바꿈). 사본을 github로 주기적으로 commit
(공유됨. 링크로 접속시에도 수정가능)

#최종갱신일 : 20201010
#최종갱신자 : 
#최종갱신내용 : 
#다음할 것 
(1) 


"""

'\n원본은 colab에서 관리 (이름 안바꿈). 사본을 github로 주기적으로 commit\n(공유됨. 링크로 접속시에도 수정가능)\n\n#최종갱신일 : 20201010\n#최종갱신자 : \n#최종갱신내용 : \n#다음할 것 \n(1) \n\n\n'

# 실행 전 설정

## 튜닝용 파라미터

In [2]:
gv_epoch = 4
gv_MAX_LEN = 128
gv_batch_size = 32    #128*64 (분)
gv_model_nm = "bert-base-multilingual-uncased"
gv_do_lower_case = True
gv_lr = 2e-5
gv_train_data = "nsmc/ratings_train.txt"

gv_num_warmup_steps = 0.2
gv_eps = 1e-8
gv_seed_val = 2019517005
gv_random_state = 20200616

## 선택한 모델 
# 88%   4 128 32    bert-base-multilingual-uncased  True  2e-5  "nsmc/ratings_train.txt" (1 epoch 당 : 27분 )   ( DFC615K 200612 #1 : 0.84999 )


#  4  64 32 'bert-base-multilingual-uncased' True  2e-5 "NLP/friends_train.json"   0.2 1e-8 2019517005 20200616 : v32   DFC615K 200617 #1 :   $$$    

## 이전 테스트 결과2
# 1  64 32    bert-base-multilingual-uncased  True  2e-5  "nsmc/ratings_train.txt" 0.2 2019517005 (1 epoch 당 : 16분 )  DFC615K 200614 #2 : 0.85052
#  1  64 32 'bert-base-multilingual-uncased' True  2e-5 "NLP/friends_train.json"   0.2 1e-8 2019517005 20200616 : v31   DFC615K 200616 #1 : 0.84087  

## 이전 테스트 결과1
# 88%   4 128 32    bert-base-multilingual-uncased  True  2e-5  "nsmc/ratings_train.txt" (1 epoch 당 : 27분 )   ( DFC615K 200612 #1 : 0.84999 )
# 88%   4 128 10    bert-base-multilingual-uncased  True  2e-5  "nsmc/ratings_train.txt" (1 epoch 당 : 34분 )
# 87%   4 128 32    bert-base-multilingual-cased    False 2e-5  "nsmc/ratings_train.txt" (1 epoch 당 : 27분 )
# 88%   8 128 32    bert-base-multilingual-uncased  True  2e-5  "nsmc/ratings_train.txt" (1 epoch 당 : 26분 )   ( DFC615K 200612 #2 : 0.84337 )
# 86%   1  64 32    bert-base-multilingual-uncased  True  2e-5  "nsmc/ratings_train.txt" (1 epoch 당 : 15분 )   ( 3회차 : 0.82406 )
# 95%   4 128 32    bert-base-multilingual-uncased  True  2e-5  "nsmc/ratings.txt"       (1 epoch 당 : 35분 ) ( DFC615K 200612 #3 : 0.82710 )
# 83%   1  32 64    bert-base-multilingual-uncased  True  2e-5  "nsmc/ratings_train.txt" (1 epoch 당 :  8분 )
# ??%   8  64 64    bert-base-multilingual-uncased  True  1e-5  "nsmc/ratings_train.txt" 0.2 (1 epoch 당 : ??분 ) ( DFC615K 200614 #1 : 0.49991 : 순서 잘못 제출 )



## Gdrive 연결 (모델 저장용)

In [3]:
from os import path
from google.colab import drive

In [4]:
model_dir_name = 'bert_model_save'
drive.mount('/content/gdrive')
model_base_dir = path.join('./gdrive/My Drive/', model_dir_name)
if not path.exists(model_base_dir):
  print('Check your google drive directory. See you file explorer')

# 20201010 : 4/1AfDhmrhGH8hkxOprj9N90M6-BB7WTft9DlV_JT-9ZuhOYHcm9B32z9QNK1Y  

Mounted at /content/gdrive


# HW 확인


## GPU 확인

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

#!nvidia-smi

## GPU 
# 기본(저) : K80 (50만원)
# 기본(고) : Tesla T4 (350만원)
# Pro (저) : Tesla T4 (350만원)  
# Pro (고) : Tesla P100-PCIE... (430만원)

## 유지시간 
# 기본 : 12시간
# Pro  : 24시간 

## RAM 
# 기본 : 12.72 GB
# Pro  : 25.51 GB (런타임 유형 고용량 RAM 변경)


Sun Oct 11 00:05:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 메모리 확인

In [6]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


## 기타 사양 확인 

In [7]:
# OS
!cat /etc/issue.net

Ubuntu 18.04.5 LTS


In [8]:
# CPU 사양
!head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.176
cache size	: 39424 KB
physical id	: 0


In [9]:
# 메모리 사양
!head -n 3 /proc/meminfo

MemTotal:       26751688 kB
MemFree:        23937276 kB
MemAvailable:   25759748 kB


In [10]:
# 디스크 사양
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         148G   31G  111G  22% /
tmpfs            64M     0   64M   0% /dev
tmpfs            13G     0   13G   0% /sys/fs/cgroup
shm              13G     0   13G   0% /dev/shm
tmpfs            13G   16K   13G   1% /var/colab
/dev/sda1       154G   32G  122G  21% /opt/bin
tmpfs            13G     0   13G   0% /proc/acpi
tmpfs            13G     0   13G   0% /proc/scsi
tmpfs            13G     0   13G   0% /sys/firmware
drive            15G  724M   15G   5% /content/gdrive


In [11]:
!python --version

Python 3.6.9


# 환경설정

## 라이브러리 설치

In [12]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 1.1MB 9.6MB/s 
     |████████████████████████████████| 3.0MB 33.3MB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
     |████████████████████████████████| 1.1MB 50.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1ac66e38d19ad6e8b700d1b968bf3304bf71084e9b1b3529c235f1a81ebb306f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## 라이브러리 import

In [13]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

from google.colab import files

# 데이터셋 준비

## raw data 준비

In [14]:
# 네이버 영화리뷰 감정분석 데이터 다운로드
!git clone https://github.com/e9t/nsmc.git

# submit 용 데이터 (개인)
!git clone https://github.com/head4ths/NLP.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 12.73 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.
Cloning into 'NLP'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 45 (delta 19), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [15]:
# 디렉토리의 파일 목록
!ls nsmc -la

total 38648
drwxr-xr-x 5 root root     4096 Oct 11 00:06 .
drwxr-xr-x 1 root root     4096 Oct 11 00:06 ..
drwxr-xr-x 2 root root     4096 Oct 11 00:06 code
drwxr-xr-x 8 root root     4096 Oct 11 00:06 .git
-rw-r--r-- 1 root root  4893335 Oct 11 00:06 ratings_test.txt
-rw-r--r-- 1 root root 14628807 Oct 11 00:06 ratings_train.txt
-rw-r--r-- 1 root root 19515078 Oct 11 00:06 ratings.txt
drwxr-xr-x 2 root root   471040 Oct 11 00:06 raw
-rw-r--r-- 1 root root     2596 Oct 11 00:06 README.md
-rw-r--r-- 1 root root    36746 Oct 11 00:06 synopses.json


## pandas load

In [16]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv(gv_train_data, sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')
submit = pd.read_csv("NLP/ko_data.csv") 

submit["label"] = 0
submit.rename(columns = {'Sentence': 'document'}, inplace = True)

print(train.shape)
print(test.shape)
print(submit.shape)

(150000, 3)
(50000, 3)
(11187, 3)


In [17]:
# 훈련셋의 앞부분 출력
train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [18]:
test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [19]:
submit.head()

,Id,document,label
0,0,정말 많이 울었던 영화입니다.,0
1,1,시간 낭비예요.,0
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.,0
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!,0
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.,0


# 전처리 - Traning Set 

## 형식 변환 [CLS] [SEP]

In [20]:
# 리뷰 문장 추출
sentences = train['document']
sentences[:10]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
5        막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
6                                원작의 긴장감을 제대로 살려내지못했다.
7    별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...
8                               액션이 없는데도 재미 있는 몇안되는 영화
9        왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?
Name: document, dtype: object

In [21]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]',
 '[CLS] 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움. [SEP]',
 '[CLS] 원작의 긴장감을 제대로 살려내지못했다. [SEP]',
 '[CLS] 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [SEP]',
 '[CLS] 액션이 없는데도 재미 있는 몇안되는 영화 [SEP]',
 '[CLS] 왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [SEP]']

In [22]:
# 라벨 추출
labels = train['label'].values
labels

array([0, 1, 0, ..., 0, 1, 0])

## BERT 토크나이징

In [23]:
# BERT의 토크나이저로 문장을 토큰으로 분리
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]
['[CLS]', 'ᄋ', '##ᅡ', '더', '##비', '##ᆼ', '.', '.', '진', '##ᄍ', '##ᅡ', 'ᄍ', '##ᅡ', '##즈', '##ᆼ', '##나', '##네', '##요', 'ᄆ', '##ᅩᆨ', '##소', '##리', '[SEP]']


In [24]:
# 입력 토큰의 최대 시퀀스 길이
#MAX_LEN = 128
#MAX_LEN = 64 #head4ths
MAX_LEN = gv_MAX_LEN

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101,  1174, 25539, 23236, 29234, 13045,   119,   119, 87550,
       97082, 25539,  1176, 25539, 24937, 13045, 16801, 72197, 47024,
        1169, 70724, 22585, 13926,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

## 어텐션 마스크

In [25]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [26]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=gv_random_state, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=gv_random_state, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  1172, 89474, 13503, 27374,  1169, 93871,  1172, 89474, 32542,
        33401, 31542,   126, 31049, 18823,  1181, 76145, 22699, 11426, 22272,
        41912,   119, 60889, 18139,  1164, 49904, 97107, 18721,  1174, 26179,
        94083, 11537, 40815, 18463, 18823, 23236, 11830, 40049, 28799, 97109,
        12300, 64067, 70870, 84766,   119,  1174, 25539, 12261, 82346, 12398,
        30555, 10609, 58957, 18253, 90834, 18992,  1172, 29347, 13926, 35132,
        72197,   119,   122, 94361, 30182, 10576, 33401, 92760,  1170, 40389,
        25536, 14280, 13045, 97072, 31496, 13503,   119,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

## 배치 사이즈

In [27]:
# 배치 사이즈
#batch_size = 256
#batch_size = 32
batch_size = gv_batch_size

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# 전처리 - Test Set 

## 형식 변환 [CLS] [SEP]

In [28]:
# 리뷰 문장 추출
sentences = test['document']
sentences[:10]

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
5                                   음악이 주가 된, 최고의 음악영화
6                                              진정한 쓰레기
7             마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다
8    갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한c...
9       이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..
Name: document, dtype: object

In [29]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 굳 ㅋ [SEP]',
 '[CLS] GDNTOPCLASSINTHECLUB [SEP]',
 '[CLS] 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 [SEP]',
 '[CLS] 지루하지는 않은데 완전 막장임... 돈주고 보기에는.... [SEP]',
 '[CLS] 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? [SEP]',
 '[CLS] 음악이 주가 된, 최고의 음악영화 [SEP]',
 '[CLS] 진정한 쓰레기 [SEP]',
 '[CLS] 마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 [SEP]',
 '[CLS] 갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 [SEP]',
 '[CLS] 이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. [SEP]']

In [30]:
# 라벨 추출
labels = test['label'].values
labels

array([1, 0, 0, ..., 0, 0, 0])

## BERT 토크나이징

In [31]:
# BERT의 토크나이저로 문장을 토큰으로 분리
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 굳 ㅋ [SEP]
['[CLS]', '구', '##ᆮ', '[UNK]', '[SEP]']


In [32]:
# 입력 토큰의 최대 시퀀스 길이
#MAX_LEN = 128
#MAX_LEN = 64 #head4ths
MAX_LEN = gv_MAX_LEN

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101, 54602, 97097,   100,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

## 어텐션 마스크

In [33]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [34]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([  101, 54602, 97097,   100,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

## 배치 사이즈

In [35]:
# 배치 사이즈
#batch_size = 256
#batch_size = 32
batch_size = gv_batch_size

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 전처리 - Submit Set

## 형식 변환 [CLS][SEP]

In [36]:
# 리뷰 문장 추출
sentences = submit['document']
sentences[:10]

0                          정말 많이 울었던 영화입니다.
1                                  시간 낭비예요.
2    포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3      지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4                 이걸 영화로 만드는 거야?얼마나 가는지 보자.
5        잔잔한 감동을 주는 영화가 좋은 영화다. 줄리안 무어의 매력!
6                   프랑스 영화, 정말 재미없다는 말밖에...
7                        이보다 더 자연스러울 수는 없다.
8                              잠만 자고 있었는데~~
9             오프닝 씬이... (이 영화와 젊음에 대해 말한다.)
Name: document, dtype: object

In [37]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 정말 많이 울었던 영화입니다. [SEP]',
 '[CLS] 시간 낭비예요. [SEP]',
 '[CLS] 포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다. [SEP]',
 '[CLS] 지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!! [SEP]',
 '[CLS] 이걸 영화로 만드는 거야?얼마나 가는지 보자. [SEP]',
 '[CLS] 잔잔한 감동을 주는 영화가 좋은 영화다. 줄리안 무어의 매력! [SEP]',
 '[CLS] 프랑스 영화, 정말 재미없다는 말밖에... [SEP]',
 '[CLS] 이보다 더 자연스러울 수는 없다. [SEP]',
 '[CLS] 잠만 자고 있었는데~~ [SEP]',
 '[CLS] 오프닝 씬이... (이 영화와 젊음에 대해 말한다.) [SEP]']

In [38]:
# 라벨 추출
labels = submit['label'].values
labels

array([0, 0, 0, ..., 0, 0, 0])

## BERT 토크나이징


In [39]:
# BERT의 토크나이저로 문장을 토큰으로 분리
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained(gv_model_nm, do_lower_case=gv_do_lower_case)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 정말 많이 울었던 영화입니다. [SEP]
['[CLS]', '정', '##말', '많이', 'ᄋ', '##ᅮᆯ', '##었던', '영화', '##입', '##니다', '.', '[SEP]']


In [40]:
# 입력 토큰의 최대 시퀀스 길이
#MAX_LEN = 128
#MAX_LEN = 64 #head4ths
MAX_LEN = gv_MAX_LEN

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101, 81463, 84766, 46290,  1174, 65633, 81356, 41912, 62776,
       82566,   119,   102,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

## 어텐션 마스크

In [41]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [42]:
# 데이터를 파이토치의 텐서로 변환
submit_inputs = torch.tensor(input_ids)
submit_labels = torch.tensor(labels)
submit_masks = torch.tensor(attention_masks)

print(submit_inputs[0])
print(submit_labels[0])
print(submit_masks[0])

tensor([  101, 81463, 84766, 46290,  1174, 65633, 81356, 41912, 62776, 82566,
          119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

## 배치 사이즈

In [43]:
# 배치 사이즈
#batch_size = 256
#batch_size = 32
batch_size = gv_batch_size

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
submit_data = TensorDataset(submit_inputs, submit_masks, submit_labels)
submit_sampler = SequentialSampler(submit_data)
submit_dataloader = DataLoader(submit_data, sampler=submit_sampler, batch_size=batch_size, shuffle=False)


# 모델 생성

## 사전 설정

In [44]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [45]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


## 모델 생성

In [46]:
# 분류를 위한 BERT 모델 생성
#model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
#model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=2)
model = BertForSequenceClassification.from_pretrained(gv_model_nm, num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [47]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (105879, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

## 옵티마이저 및 에폭 설정

In [48]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = gv_lr, # 학습률
                  eps = gv_eps # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = gv_epoch

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = gv_num_warmup_steps,
                                            num_training_steps = total_steps)

# 모델 학습 - Traning Set

## 함수 정의

In [49]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [50]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

## 학습 수행

In [51]:
# 21:45 ~ 21:57 ~22:17
# 약 20분 
# head4ths
# epoch : 4 > 1
# MAX length : 128 > 64

# 재현을 위해 랜덤시드 고정
seed_val = gv_seed_val
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

## T4 : 50분

## P100
# 64 len : 에포크 당 12분  (batch : 32)
# 128 len : 에포크 당 30분  (batch : 32)

## V100 (P100보다 좋음)
# 128 len : 에포크 당 16분  (batch : 32)


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:01:54.
  Batch 1,000  of  4,219.    Elapsed: 0:03:48.
  Batch 1,500  of  4,219.    Elapsed: 0:05:42.
  Batch 2,000  of  4,219.    Elapsed: 0:07:37.
  Batch 2,500  of  4,219.    Elapsed: 0:09:31.
  Batch 3,000  of  4,219.    Elapsed: 0:11:25.
  Batch 3,500  of  4,219.    Elapsed: 0:13:19.
  Batch 4,000  of  4,219.    Elapsed: 0:15:12.

  Average training loss: 0.374
  Training epcoh took: 0:16:02

Running Validation...
  Accuracy: 0.864
  Validation took: 0:00:32

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:01:54.
  Batch 1,000  of  4,219.    Elapsed: 0:03:48.
  Batch 1,500  of  4,219.    Elapsed: 0:05:41.
  Batch 2,000  of  4,219.    Elapsed: 0:07:35.
  Batch 2,500  of  4,219.    Elapsed: 0:09:29.
  Batch 3,000  of  4,219.    Elapsed: 0:11:23.
  Batch 3,500  of  4,219.    Elapsed: 0:13:17.
  Batch 4,000  of  4,219.    Elapsed: 0:15:11.

  Average training loss: 0.276

# 모델 Gdrive 에 저장 

In [52]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/gdrive/My Drive/bert_model_save/'

# Create output directory if needed
"""
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
"""

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to /content/gdrive/My Drive/bert_model_save/


('/content/gdrive/My Drive/bert_model_save/vocab.txt',
 '/content/gdrive/My Drive/bert_model_save/special_tokens_map.json',
 '/content/gdrive/My Drive/bert_model_save/added_tokens.json')

In [53]:
# 미국 시간으로 나옴. 실제시간보다 9시간 느리게 나옴 (한국 10:00, 미국 01:00)
!ls -l --block-size=K /content/gdrive/'My Drive'/bert_model_save

total 654682K
-rw------- 1 root root      1K Oct 11 01:14 config.json
-rw------- 1 root root 653828K Oct 11 01:15 pytorch_model.bin
-rw------- 1 root root      1K Oct 11 01:15 special_tokens_map.json
-rw------- 1 root root      1K Oct 11 01:15 tokenizer_config.json
-rw------- 1 root root    852K Oct 11 01:15 vocab.txt


In [54]:
!ls -l --block-size=M /content/gdrive/'My Drive'/bert_model_save/pytorch_model.bin

-rw------- 1 root root 639M Oct 11 01:15 '/content/gdrive/My Drive/bert_model_save/pytorch_model.bin'


In [55]:
"""
# 모델 불러오는 법
# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)
"""

'\n# 모델 불러오는 법\n# Load a trained model and vocabulary that you have fine-tuned\nmodel = BertForSequenceClassification.from_pretrained(output_dir)\ntokenizer = BertTokenizer.from_pretrained(output_dir)\n\n# Copy the model to the GPU.\nmodel.to(device)\n'

# 평가 - Submit Set

In [56]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

#csv
submit_preds =  []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(submit_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(submit_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

    #csv  
    submit_preds.extend(logits.argmax(-1))         

print("")
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
print("Submit took: {:}".format(format_time(time.time() - t0)))


  Batch   100  of    350.    Elapsed: 0:00:07.
  Batch   200  of    350.    Elapsed: 0:00:14.
  Batch   300  of    350.    Elapsed: 0:00:20.

Accuracy: 0.485
Submit took: 0:00:24


# 평가 - Test Set 

In [57]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


  Batch   100  of  1,563.    Elapsed: 0:00:07.
  Batch   200  of  1,563.    Elapsed: 0:00:14.
  Batch   300  of  1,563.    Elapsed: 0:00:20.
  Batch   400  of  1,563.    Elapsed: 0:00:27.
  Batch   500  of  1,563.    Elapsed: 0:00:34.
  Batch   600  of  1,563.    Elapsed: 0:00:41.
  Batch   700  of  1,563.    Elapsed: 0:00:47.
  Batch   800  of  1,563.    Elapsed: 0:00:54.
  Batch   900  of  1,563.    Elapsed: 0:01:01.
  Batch 1,000  of  1,563.    Elapsed: 0:01:08.
  Batch 1,100  of  1,563.    Elapsed: 0:01:14.
  Batch 1,200  of  1,563.    Elapsed: 0:01:21.
  Batch 1,300  of  1,563.    Elapsed: 0:01:28.
  Batch 1,400  of  1,563.    Elapsed: 0:01:35.
  Batch 1,500  of  1,563.    Elapsed: 0:01:41.

Accuracy: 0.882
Test took: 0:01:46


# 시연

## 시연용 함수

In [58]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    #MAX_LEN = 128
    #MAX_LEN = 64 #head4ths
    MAX_LEN = gv_MAX_LEN

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [59]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

## 시연 수행

In [60]:
logits = test_sentences(['정말 많이 울었던 영화입니다.'])
print(logits)
print(np.argmax(logits))

[[-2.6290166  2.3457255]]
1


In [61]:
logits = test_sentences(['시간 낭비예요.'])
print(logits)
print(np.argmax(logits))

[[ 3.4735072 -3.020334 ]]
0


In [62]:
logits = test_sentences(['포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.'])
print(logits)
print(np.argmax(logits))

[[ 1.3983015 -1.292294 ]]
0


In [63]:
logits = test_sentences(['지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!'])
print(logits)
print(np.argmax(logits))

[[-3.0665598  2.8420155]]
1


In [64]:
logits = test_sentences(['이걸 영화로 만드는 거야?얼마나 가는지 보자.'])
print(logits)
print(np.argmax(logits))

[[ 2.9791198 -2.4930468]]
0


# 캐글용 csv 생성

In [65]:
#submit_pred_labels = [labels_num[submit_pred] for submit_pred in submit_preds]

In [66]:
"""
#submission_dic = {"id":ids, "Predicted":submit_pred_labels} 
submission_dic = {"id":list(range(len(submit_preds))), "Predicted":submit_preds} 

submission_df = pd.DataFrame(submission_dic) 
submission_df.to_csv("DFC615K 200619 A11.csv", index=False)
files.download("DFC615K 200619 A11.csv")

# https://stackoverflow.com/questions/53581023/google-colab-file-download-failed-to-fetch-error
# you need to enable third-party cookies but for now it only works for Chrome browser, open
# chrome://settings/content/cookies
# make sure the option for Block third-party cookies is disabled and click add button in Allow section then add
# colab.research.google.com
"""

'\n#submission_dic = {"id":ids, "Predicted":submit_pred_labels} \nsubmission_dic = {"id":list(range(len(submit_preds))), "Predicted":submit_preds} \n\nsubmission_df = pd.DataFrame(submission_dic) \nsubmission_df.to_csv("DFC615K 200619 A11.csv", index=False)\nfiles.download("DFC615K 200619 A11.csv")\n\n# https://stackoverflow.com/questions/53581023/google-colab-file-download-failed-to-fetch-error\n# you need to enable third-party cookies but for now it only works for Chrome browser, open\n# chrome://settings/content/cookies\n# make sure the option for Block third-party cookies is disabled and click add button in Allow section then add\n# colab.research.google.com\n'